In [ ]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("ALS").getOrCreate()

In [3]:
from pyspark import SparkFiles
# read cons.csv file into spark DataFrame
url = "https://storage.googleapis.com/sharp-matter-286015.appspot.com/cons.csv"
spark.sparkContext.addFile(url)
df_cons = spark.read.csv(SparkFiles.get("cons.csv"), sep=",", header=True)
#df_cons = spark.read.csv("../put csv here/cons.csv",header=True)
#df = spark.read.format("csv").load("cons.csv")

# # select required columns and alias as per requirement
df_cons = df_cons.select(col("cons_id"), col("create_dt"),col("modified_dt").alias("updated_dt")\
                          , col("subsource").alias("code"))

# # Show DataFrame
df_con.show(n=2)
type(df_cons)

+-------+------+---------+----------+--------+------+--------------------+------+---------------+--------------------+--------+----------+----------------+--------+--------------------+------+----------+------------+---------+--------------------------+---------------+--------------------+----------+-----------+--------------------+------------+-------------+------+----+
|cons_id|prefix|firstname|middlename|lastname|suffix|          salutation|gender|       birth_dt|               title|employer|occupation|          income|  source|           subsource|userid|  password|is_validated|is_banned|change_password_next_login|consent_type_id|           create_dt|create_app|create_user|         modified_dt|modified_app|modified_user|status|note|
+-------+------+---------+----------+--------+------+--------------------+------+---------------+--------------------+--------+----------+----------------+--------+--------------------+------+----------+------------+---------+--------------------------

In [4]:
# read cons_email.csv file into spark DataFrame
url = "https://storage.googleapis.com/sharp-matter-286015.appspot.com/cons_email.csv"
spark.sparkContext.addFile(url)
df_cons_email = spark.read.csv(SparkFiles.get("cons_email.csv"), sep=",", header=True)

#df_cons_email = spark.read.csv("../put csv here/cons_email.csv",header=True)
# select required columns and alias as per requirement
df_cons_email = df_cons_email.select(col("cons_id"), col("cons_email_id"),col("email"), col("status"))
df_cons_email.show(n=2)

+-------+-------------+--------------------+------+
|cons_id|cons_email_id|               email|status|
+-------+-------------+--------------------+------+
| 548198|            1|xmartinez@vincent...|     1|
| 491137|            2|  hmiller@haynes.biz|     1|
+-------+-------------+--------------------+------+
only showing top 2 rows



In [6]:
# read cons_email.csv file into spark DataFrame
url = "https://storage.googleapis.com/sharp-matter-286015.appspot.com/cons_email_chapter_subscription.csv"
spark.sparkContext.addFile(url)
df_cons_email_chapter_subscription_email = spark.read.csv(SparkFiles.get("cons_email_chapter_subscription.csv"), sep=",", header=True)
#df_cons_email_chapter_subscription_email = spark.read.csv("../put csv here/cons_email_chapter_subscription.csv",header=True)
df_cons_email_chapter_subscription_email=df_cons_email_chapter_subscription_email.\
filter(df_cons_email_chapter_subscription_email.chapter_id==1)
df_cons_email_chapter_subscription_email.show(n=2)

+----------------------------------+-------------+----------+-------+--------------------+--------------------+
|cons_email_chapter_subscription_id|cons_email_id|chapter_id|isunsub|            unsub_dt|         modified_dt|
+----------------------------------+-------------+----------+-------+--------------------+--------------------+
|                                 1|       332188|         1|      1|Sat, 1971-06-12 1...|Thu, 1990-06-28 1...|
|                                 2|       536526|         1|      1|Wed, 2006-07-12 0...|Thu, 1979-09-20 0...|
+----------------------------------+-------------+----------+-------+--------------------+--------------------+
only showing top 2 rows



In [7]:
df_join_cons_id = df_cons.join(df_cons_email, on=['cons_id'], how='inner')
df_join_cons_id.show(n=2)
df_join_cons_id.count()

+-------+--------------------+--------------------+--------------------+-------------+--------------------+------+
|cons_id|           create_dt|          updated_dt|                code|cons_email_id|               email|status|
+-------+--------------------+--------------------+--------------------+-------------+--------------------+------+
| 100010|Sat, 2019-09-14 1...|Sat, 2013-11-23 2...|                null|       630340|stephanie18@cox.info|     1|
| 100140|Thu, 1999-09-23 0...|Mon, 1987-03-02 1...|pUXsobfAnPYGUGbOMEtz|       289643|tracirich@hickman...|     1|
+-------+--------------------+--------------------+--------------------+-------------+--------------------+------+
only showing top 2 rows



1400000

In [9]:
df_join_cons_email_id = df_join_cons_id.join(df_cons_email_chapter_subscription_email, on=['cons_email_id'], how='inner')
df_join_cons_email_id.show(n=2)
df_join_cons_email_id.count()

+-------------+-------+--------------------+--------------------+--------------------+--------------------+------+----------------------------------+----------+-------+--------+-----------+
|cons_email_id|cons_id|           create_dt|          updated_dt|                code|               email|status|cons_email_chapter_subscription_id|chapter_id|isunsub|unsub_dt|modified_dt|
+-------------+-------+--------------------+--------------------+--------------------+--------------------+------+----------------------------------+----------+-------+--------+-----------+
|       100010| 386479|Mon, 1972-01-17 1...|Thu, 2011-06-16 2...|DAQWqTYFyOUiyuMTCLGi| scottcody@yahoo.com|     1|                              null|      null|   null|    null|       null|
|      1000240| 188649|Fri, 1979-12-07 0...|Wed, 1985-07-24 2...|                null|collinsjuan@hotma...|     1|                              null|      null|   null|    null|       null|
+-------------+-------+--------------------+------

1400000

In [17]:
# write data to CSV file 
df_join_cons_email_id.coalesce(1).write.csv('people')

In [10]:
#split date coulmn and only pic date remove time and day
from pyspark.sql.functions import split
df_join_cons_email_id_date=df_join_cons_email_id.withColumn("create_dt", split(col("create_dt"), " ").getItem(1))\
.withColumn("updated_dt", split(col("updated_dt"), " ").getItem(1))

In [11]:
#convert created date and updated date to date time format
df_join_cons_email_id_type = df_join_cons_email_id_date.selectExpr("email", "code",\
    "cast(isunsub as boolean) is_unsub",\
    "cast(create_dt as date) create_dt","cast(updated_dt as date) updated_dt")

In [12]:
df_join_cons_email_id_type.printSchema()
df_join_cons_email_id_type.show()

root
 |-- email: string (nullable = true)
 |-- code: string (nullable = true)
 |-- is_unsub: boolean (nullable = true)
 |-- create_dt: date (nullable = true)
 |-- updated_dt: date (nullable = true)

+--------------------+--------------------+--------+----------+----------+
|               email|                code|is_unsub| create_dt|updated_dt|
+--------------------+--------------------+--------+----------+----------+
|stephanie18@cox.info|                null|    null|2019-09-14|2013-11-23|
|tracirich@hickman...|pUXsobfAnPYGUGbOMEtz|    null|1999-09-23|1987-03-02|
|hoodjeffrey@gmail...|mayHRqXIkQpNFXCfEEbA|    true|1992-09-26|1971-05-09|
|zhubbard@turner-b...|mayHRqXIkQpNFXCfEEbA|    null|1992-09-26|1971-05-09|
|cortezamanda@gmai...|ZtNdsqNSaDggTuWOyBDS|    true|2019-09-21|1992-01-19|
|meadowsdana@velez...|ZtNdsqNSaDggTuWOyBDS|    null|2019-09-21|1992-01-19|
|john60@callahan-c...|ZtNdsqNSaDggTuWOyBDS|    null|2019-09-21|1992-01-19|
|  thomas22@yahoo.com|QdwUCfcdJmRPaPFgymhB|    null

In [13]:
import pyspark.sql.functions as func
df_join_cons_email_id_type_group=df_join_cons_email_id_type.groupBy("create_dt")\
.agg(func.count(func.lit(1)).alias("Number of constituents"))

In [14]:
df_join_cons_email_id_type_group.show()

+----------+----------------------+
| create_dt|Number of constituents|
+----------+----------------------+
|2006-05-21|                    97|
|1983-03-12|                    82|
|1991-03-26|                    78|
|2009-11-22|                    80|
|2000-07-03|                    99|
|2015-05-19|                    71|
|1999-08-14|                    93|
|2002-06-20|                    63|
|1971-04-09|                    69|
|1983-07-07|                    78|
|1991-06-13|                    67|
|1973-05-23|                    79|
|1991-11-09|                    90|
|1995-12-01|                    53|
|1980-05-22|                    56|
|1984-07-06|                    79|
|2014-09-26|                    59|
|1977-11-06|                    76|
|2018-05-28|                    84|
|1987-07-08|                    77|
+----------+----------------------+
only showing top 20 rows



In [16]:
df_join_cons_email_id_type_group.coalesce(1).write.csv('acquisition_facts')